In [2]:
from suite2p import run_s2p
import json
from pathlib import Path
import numpy as np

import tables
import suite2p
import shutil
from tqdm import tqdm
import flammkuchen as fl
from tifffile import imwrite
import hdf5plugin

In [3]:
master_path = Path(r"\\Funes\Shared\experiments\E0040_motions_cardinal\v13_cw_ccw\ls_fixed\spont_plus_v13\h2b\new")
master_path = Path(r"\\funes\Shared\experiments\E0040_motions_cardinal\v19_clolclol\ls\wb_gad1b")

fast_path_dest = Path(r"D:\Imaging") 
files = list(master_path.glob("*_f*"))

unprocessed = [f for f in files if not (f / "suite2p" / "combined").exists()]
print(unprocessed)
print(len(unprocessed))

[WindowsPath('//funes/Shared/experiments/E0040_motions_cardinal/v19_clolclol/ls/wb_gad1b/22060111_f2'), WindowsPath('//funes/Shared/experiments/E0040_motions_cardinal/v19_clolclol/ls/wb_gad1b/220601_f0')]
2


In [4]:
for i, path in enumerate(tqdm(unprocessed)):
    print("#####################################")
    print(path)
    dataset_path = path / "original"
    
    fast_path = fast_path_dest / f"suite{i}p"
    fast_path.mkdir(exist_ok=True)

    with open(dataset_path / "stack_metadata.json") as f:
        stack_metadata = json.load(f)
    
    try:
        with open(path / "*metadata.json") as f:
            metadata = json.load(f)

        fs = int(metadata["imaging"]["microscope_config"]["lightsheet"]["scanning"]["z"]["frequency"]) # sampling frequency
    except:
        fs = 3
    n_planes = stack_metadata["shape_full"][1]  # number of planes
    
    ops = {
        # file paths
        'look_one_level_down': False, # whether to look in all subfolders when searching for tiffs
        'fast_disk': str(fast_path), # used to store temporary binary file, defaults to save_path0
        'delete_bin': True, # whether to delete binary file after processing
        'mesoscan': False, # for reading in scanimage mesoscope files
        'bruker': False, # whether or not single page BRUKER tiffs!
        'h5py': str(dataset_path), # take h5py as input (deactivates data_path)
        'h5py_key': 'stack_4D', #key in h5py where data array is stored
        'save_path0': [], # stores results, defaults to first item in data_path
        'save_folder': [],
        'subfolders': [],
        'move_bin': True, # if 1, and fast_disk is different than save_disk, binary file is moved to save_disk
        
        # main settings
        'nplanes' : n_planes, # each tiff has these many planes in sequence
        'nchannels' : 1, # each tiff has these many channels per plane
        'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
        'tau':  1.25, # this is the main parameter for deconvolution
        'fs': fs,  # sampling rate (PER PLANE e.g. for 12 plane recordings it will be around 2.5)
        'force_sktiff': False, # whether or not to use scikit-image for tiff reading
        'frames_include': -1,
        "num_workers": 10,
        
        # output settings
        'preclassify': 0., # apply classifier before signal extraction with probability 0.3
        'save_mat': False, # whether to save output as matlab files
        'combined': True, # combine multiple planes into a single result /single canvas for GUI
        'aspect': 1.0, # um/pixels in X / um/pixels in Y (for correct aspect ratio in GUI)
        
        # bidirectional phase offset
        'do_bidiphase': False,
        'bidiphase': 0,
        'bidi_corrected': False,
        
        # registration settings
        'do_registration': 1, # whether to register data (2 forces re-registration)
        'two_step_registration': False,
        'keep_movie_raw': False,
        'nimg_init': 300, # subsampled frames for finding reference image
        'batch_size': 500, # number of frames per batch
        'maxregshift': 0.1, # max allowed registration shift, as a fraction of frame max(width and height)
        'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
        'reg_tif': False, # whether to save registered tiffs
        'reg_tif_chan2': False, # whether to save channel 2 registered tiffs
        'subpixel' : 5, # precision of subpixel registration (1/subpixel steps)
        'smooth_sigma_time' : 0, # gaussian smoothing in time
        'smooth_sigma': 1.15, # ~1 good for 2P recordings, recommend >5 for 1P recordings
        'th_badframes': 1.0, # this parameter determines which frames to exclude when determining cropping - set it smaller to exclude more frames
        'pad_fft': False,
        
        # non rigid registration settings
        'nonrigid': False, # whether to use nonrigid registration
        'block_size': [128, 128], # block size to register (** keep this a multiple of 2 **)
        'snr_thresh': 1.2, # if any nonrigid block is below this threshold, it gets smoothed until above this threshold. 1.0 results in no smoothing
        'maxregshiftNR': 5, # maximum pixel shift allowed for nonrigid, relative to rigid
        
         # 1P settings
        '1Preg': False, # whether to perform high-pass filtering and tapering
        'spatial_hp': 25, # window for spatial high-pass filtering before registration
        'pre_smooth': 2, # whether to smooth before high-pass filtering before registration
        'spatial_taper': 50, # how much to ignore on edges (important for vignetted windows, for FFT padding do not set BELOW 3*ops['smooth_sigma'])
        
        # cell detection settings
        'roidetect': True, # whether or not to run ROI extraction
        'sparse_mode': False, # whether or not to run sparse_mode
        'diameter': 6, # if not sparse_mode, use diameter for filtering and extracting
        'spatial_scale': 0, # 0: multi-scale; 1: 6 pixels, 2: 12 pixels, 3: 24 pixels, 4: 48 pixels
        'connected': True, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
        'nbinned': 5000, # max number of binned frames for cell detection
        'max_iterations': 20, # maximum number of iterations to do cell detection
        'threshold_scaling': 1
        , # adjust the automatically determined threshold by this scalar multiplier
        'max_overlap': 0.9, # cells with more overlap than this get removed during triage, before refinement
        'high_pass': 80, # running mean subtraction with window of size 'high_pass' (use low values for 1P)
        
        # ROI extraction parameters
        'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
        'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
        'allow_overlap': False, # pixels that are overlapping are thrown out (False) or added to both ROIs (True)
    
        # channel 2 detection settings (stat[n]['chan2'], stat[n]['not_chan2'])
        'chan2_thres': 0.65, # minimum for detection of brightness on channel 2
    
        # deconvolution settings
        'spikedetect': False, # whether or not to run spike deconvolution
        'baseline': 'maximin', # baselining mode (can also choose 'prctile')
        'win_baseline': 60., # window for maximin
        'sig_baseline': 10., # smoothing constant for gaussian filter
        'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
        'neucoeff': .7,  # neuropil coefficient
        'xrange': np.array([0, 0]),
        'yrange': np.array([0, 0]),
    }
    
    db = {'data_path': [str(dataset_path)]}
    
    try:
        opsEnd=run_s2p(ops=ops, db=db)
    except (UnboundLocalError, ValueError):
        print("Error for fish ", path)
    
    shutil.rmtree(fast_path)

  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

#####################################
\\funes\Shared\experiments\E0040_motions_cardinal\v19_clolclol\ls\wb_gad1b\22060111_f2
{'data_path': ['\\\\funes\\Shared\\experiments\\E0040_motions_cardinal\\v19_clolclol\\ls\\wb_gad1b\\22060111_f2\\original']}
h5
** Found 249 h5 files - converting to binary **
NOTE: using a list of h5 files:
['\\\\funes\\Shared\\experiments\\E0040_motions_cardinal\\v19_clolclol\\ls\\wb_gad1b\\22060111_f2\\original\\0000.h5', '\\\\funes\\Shared\\experiments\\E0040_motions_cardinal\\v19_clolclol\\ls\\wb_gad1b\\22060111_f2\\original\\0001.h5', '\\\\funes\\Shared\\experiments\\E0040_motions_cardinal\\v19_clolclol\\ls\\wb_gad1b\\22060111_f2\\original\\0002.h5', '\\\\funes\\Shared\\experiments\\E0040_motions_cardinal\\v19_clolclol\\ls\\wb_gad1b\\22060111_f2\\original\\0003.h5', '\\\\funes\\Shared\\experiments\\E0040_motions_cardinal\\v19_clolclol\\ls\\wb_gad1b\\22060111_f2\\original\\0004.h5', '\\\\funes\\Shared\\experiments\\E0040_motions_cardinal\\v19_clolclol\\ls\\w

c:\users\lavian\anaconda3\envs\suite2p\lib\site-packages\suite2p\registration\register.py:45: RuntimeWarning: invalid value encountered in true_divide
  dxy = dxy / dxy.mean()
c:\users\lavian\anaconda3\envs\suite2p\lib\site-packages\suite2p\registration\register.py:50: RuntimeWarning: invalid value encountered in greater
  badframes = np.logical_or(px > th_badframes * 100, badframes)


added enhanced mean image
----------- Total 518.22 sec
Registration metrics, 142.56 sec.
NOTE: applying default C:\Users\lavian\.suite2p\classifiers\classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 04
Binned movie [1750,648,1024] in 120.87 sec.
ROIs: 200, cost: 0.5321, time: 144.6204
ROIs: 400, cost: 0.5303, time: 207.1028
ROIs: 600, cost: 0.5287, time: 348.4133
ROIs: 800, cost: 0.5269, time: 511.4250
ROIs: 1000, cost: 0.5251, time: 717.7948
ROIs: 1200, cost: 0.5233, time: 963.8199
ROIs: 1400, cost: 0.5213, time: 1203.5159
ROIs: 1600, cost: 0.5194, time: 1410.0730
ROIs: 1800, cost: 0.5176, time: 1607.6779
ROIs: 2000, cost: 0.5159, time: 1811.9228
ROIs: 2200, cost: 0.5144, time: 1971.8263
ROIs: 2400, cost: 0.5131, time: 2104.9931
ROIs: 2600, cost: 0.5119, time: 2233.7242
ROIs: 2768, cost: 0.5108, time: 2364.6579
ROIs: 2831, cost: 0.5101, time: 2501.7172
ROIs: 2874, cost: 0.5095, time: 2629.0980
ROIs: 2919, cost: 0.5089, time: 2762.7446
ROIs: 2961, cost: 0.

c:\users\lavian\anaconda3\envs\suite2p\lib\site-packages\suite2p\extraction\extract.py:112: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  Fi[n] = np.dot(data[:, cell_ipix[n]], cell_lam[n])


...

c:\users\lavian\anaconda3\envs\suite2p\lib\site-packages\suite2p\detection\sourcery.py:283: RuntimeWarning: Mean of empty slice
  footprints[k] = np.nanmean(rs[proj > proj.max() * frac])


Detected 3352 ROIs, 2765.19 sec
After removing overlaps, 3286 ROIs remain
----------- Total 2830.30 sec.
----------- EXTRACTION
Masks created, 13.66 sec.
Extracted fluorescence from 3286 ROIs in 7200 frames, 42.07 sec.
----------- Total 60.49 sec.
----------- CLASSIFICATION
['compact', 'skew', 'npix_norm']
----------- Total 0.06 sec.
moving binary files to save_path
Plane 0 processed in 3737.54 sec (can open in GUI).
>>>>>>>>>>>>>>>>>>>>> PLANE 1 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 7200 frames
Reference frame, 40.83 sec.
Registered 2000/7200 in 85.61s
Registered 4000/7200 in 169.47s
Registered 6000/7200 in 253.86s
added enhanced mean image
----------- Total 356.10 sec
Registration metrics, 77.15 sec.
NOTE: applying default C:\Users\lavian\.suite2p\classifiers\classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 04
Binned movie

c:\users\lavian\anaconda3\envs\suite2p\lib\site-packages\suite2p\detection\sourcery.py:283: RuntimeWarning: Mean of empty slice
  footprints[k] = np.nanmean(rs[proj > proj.max() * frac])


Detected 3065 ROIs, 2166.49 sec
After removing overlaps, 2995 ROIs remain
----------- Total 2226.79 sec.
----------- EXTRACTION
Masks created, 11.71 sec.
Extracted fluorescence from 2995 ROIs in 7200 frames, 38.37 sec.
----------- Total 54.55 sec.
----------- CLASSIFICATION
['compact', 'skew', 'npix_norm']
----------- Total 0.06 sec.
moving binary files to save_path
Plane 3 processed in 3121.10 sec (can open in GUI).
>>>>>>>>>>>>>>>>>>>>> PLANE 4 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
----------- REGISTRATION
registering 7200 frames
Reference frame, 40.39 sec.
Registered 2000/7200 in 82.53s
Registered 4000/7200 in 166.41s
Registered 6000/7200 in 248.12s
added enhanced mean image
----------- Total 348.29 sec
Registration metrics, 74.07 sec.
NOTE: applying default C:\Users\lavian\.suite2p\classifiers\classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 04
Binned movie

100%|██████████████████████████████████████████████████████████████████████████████| 2/2 [24:00:30<00:00, 43215.38s/it]


In [ ]:
print(np.__version__)